# Calculating the Mandelbrot Set on a NodeMcu

Writing code to calculate the Mandelbrot Set is usually one of the first things I do when starting to work with a new platform or hardware. It helps me a lot to get in touch with the capabilities and performance of the new hardware. Probably everyone has one of these standards to start playing with. For me it's timing the duration of calculating this specific fractal.

So right after flashing the NodeMcu with the MicroPython image and being successful in turning the onboard led on and off it was about time to start playing with this.

It is impressive every single time to see that such a complete and powerful language like Python is able to run on such an tiny and cheap platform, and that it's capable of executing quite complex tasks. However the Python implementation is not feature complete and hardware resources are quite limited. Therefore a more simple and more classic approach to implement the code was necessary. I hope it's still somewhat fun to try it and play with it. Due to the limited resources the most prominent reduction  is to not generate a large colourful image, but an ascii art like view of a rather coarse grid, pretty much like the first plots of the Mandelbrot Set must have looked like.

In [1]:
import time
import machine

The ESP chip can be overclocked to twice its usual operating frequency of 80 MHz. To gain some performance from the very beginning, we adjust the frequency to 160 MHz.

In [2]:
machine.freq(160000000)
print(machine.freq())

160000000


To give a least some visual feedback from the board, we let the onboard LED flash everytime a pixel was decided to be part of the Mandelbrot Set for a very short time period. Driving the onboard LED could either be done by switching Pin 2, to which the LED is connected, to high for a moment, or by adjusting the duty cycle when im PWM mode. It turned out that the latter approach was much more reliable and faster, so the PWM mode is used for creating the visual feedback.

In [3]:
led = machine.PWM(machine.Pin(2), freq=1000)

def flash(duration=100):
    led.duty(0)
    time.sleep_us(duration)
    led.duty(1024)

flash(1000)

We will use sort of a standard set of coordinates for the real and imaginary plane to create the view. The resolution is chosen to let the output fit nicely in a jupyter notebook and keep the aspect ratio as correct as possible. Furthermore as the grid has been chosen to be quite coarse, the maximumn number of iterations is also chosen to be a rather small value, as higher values wouldn't leed to more visible detail in this case and would therefore waste a lot of computation time.

In [4]:
xl=-2.0
xr=.5
yb=-1.25
yt=1.25

width=106
height=55
maxiterations=20

dx=(xr-xl)/(width-1)
dy=(yt-yb)/(height-1)

The iteration formula itself has been encapsulated in this function. It is called for every coordinate in the complex plane defned by our coordinate window. As you can see no complex math has been used for the implementation. The real number based arithmetics has not been prefered for performance reasons. Complex math is simply currently not available in MicroPython. This leads to an even more classic implementation of the code.

In [5]:
def mandel(creal, cimag, maxiterations):
    real=creal
    imag=cimag
    for iteration in range(maxiterations):
        real2=real**2
        imag2=imag**2
        if real2+imag2 > 4.0:
            return iteration
        imag=2*real*imag+cimag
        real=real2-imag2+creal
    return maxiterations

The following code is used to loop over the coordinate plane and perform the iteration scheme at each point. The result will be a multiline string which can simply be printed out to create the famous view. If a point was found to be a member of the Mandelbrot Set a `#` is added to the string and the LED flash is triggered to give visual feedback. If that is not the case a whitespace character is used to fill the next position of the string. At the end of each row of coordinate pairs a newline charachter is added, resulting in a matrix like final view of the Mandelbrot Set.

In [6]:
tic=time.time()

rows=""
for y_index in range(height):
    row=""
    y=yb+y_index*dy
    for x_index in range(width):
        x=xl+x_index*dx
        iterations=mandel(x, y, maxiterations)
        if iterations==maxiterations:
            row+="#"
            flash()
        else:
            row+=" "
    rows+=(row+"\n")
    
toc=time.time()
print("Calculating the Mandelbrot set took {0:g} s.".format(toc-tic))

Calculating the Mandelbrot set took 12 s.


Finally we're able to present the result.

In [7]:
print(rows)

                                                                                                          
                                                                                                          
                                                                                                          
                                                                                                          
                                                                                                          
                                                                              #                           
                                                                                 #                        
                                                                               ##                         
                                                                              ####                        
                                     